In [273]:
# Overleaf
# https://www.overleaf.com/9836375457csrqydskhcmj

# Init

In [274]:
# Libraries
import numpy as np
from scipy.stats import norm

In [275]:
# Option Characteristics
S0 = 100
K = 99
r = 0.06
sigma = 0.2
T = 1.0
M = 10_000
np.random.seed = 42

# Monte Carlo

In [276]:
# Finds value of an underlying at strike
def mc_value(S0, r, sigma, T, M):
    dt = T/1    # N=1 Because of Geometric Brownian-Motion we can just simulate the variables at the final Time Step as Brownian Motion scales with time and independent increments.
    results = np.zeros(M)
    # Monte-Carlo Runs
    for i in range(M):
        S = S0 * np.exp((r - 0.5 * sigma**2) * dt + sigma * np.sqrt(dt) * np.random.normal())
        results[i] = S
    return results

# Finds value of option at strike
def mc_payoff(K, results, call, payoff_type):
    # Find option payoffs
    if payoff_type == "vanilla":
        if call:
            V = np.maximum(results - K, 0)
        else:
            V = np.maximum(K - results, 0)
    if payoff_type == "digital":
        if call:
            V = np.maximum(results - K, 0)
            V[V != 0] = 1
        else:
            V = np.maximum(K - results, 0)
            V[V != 0] = 1
    return V

# Finding Fair Value from Monte Carlo Simulation
mc_values = mc_value(S0, r, sigma, T, M)
mc_payoffs = mc_payoff(K, mc_values, call = True, payoff_type = "vanilla")
option_price = np.exp(-r * T) * np.mean(mc_payoffs)     # We have to discount the payoffs to calculate fair option's price
print(f"Option price: {option_price:.4f}")

# Finding Confidence Interval
standard_error = np.std(mc_payoffs, ddof=1) / np.sqrt(len(mc_payoffs))
t_stat = np.abs(norm.ppf(0.025))
margin_of_error = t_stat * standard_error
lower_bound = option_price - margin_of_error
upper_bound = option_price + margin_of_error
print(f"Standard error: {standard_error:.4f}")
print(f"95% Confidence interval: ({lower_bound:.4f}, {upper_bound:.4f})")

Option price: 11.6170
Standard error: 0.1622
95% Confidence interval: (11.2992, 11.9349)


# Approximating Delta

In [277]:
# Finite Difference Method

# Finding New Prices
finite_difference_step = 0.1       # Not sure how to optimize this
delta_S = S0 * finite_difference_step
S_up = S0 + delta_S
S_down = S0 - delta_S
# Finding Approxiamation for Higher Price
mc_values = mc_value(S_up, r, sigma, T, M)
mc_results_up = mc_payoff(K, mc_values, call = True, payoff_type = "vanilla")
option_price_up = np.exp(-r * T) * np.mean(mc_results_up)
# Finding Approxiamation for Lower Price
mc_values = mc_value(S_down, r, sigma, T, M)
mc_results_down = mc_payoff(K, mc_values, call = True, payoff_type = "vanilla")
option_price_down = np.exp(-r * T) * np.mean(mc_results_down)
# Approximation of Delta with Finite Differences
delta_approx = (np.mean(option_price_up) - np.mean(option_price_down)) / (2 * delta_S)

# Print the results
print(f"Delta approximation: {delta_approx:.4f}")

Delta approximation: 0.6523


# A

In [278]:
# Pathwise
delta_pathwise = np.exp(-r * T) * option_price/ S0
print(f"Delta approximation pathwise: {delta_pathwise:.4f}")

# Likelihood
d = (np.log(S0/K) + (r + 0.5*sigma**2)*T) / (sigma*np.sqrt(T))
delta_likelihood_ratio = np.mean(mc_payoffs * norm.cdf(d)) / S0
print(f"Delta approximation likelihood: {delta_likelihood_ratio:.4f}")


Delta approximation pathwise: 0.1162
Delta approximation likelihood: 0.0831


In [279]:

def delta_likelihood_ratio(S, K, r, sigma, T, N):
    """
    S: initial stock price
    K: strike price
    r: risk-free interest rate
    sigma: volatility of the stock price
    T: time to maturity
    N: number of simulations
    """
    # Calculate required parameters
    dt = T/N
    d1 = (np.log(S/K) + (r + 0.5*sigma**2)*T) / (sigma*np.sqrt(T))
    
    # Generate random numbers for stock price simulations
    Z = np.random.standard_normal(N)
    S_t = S * np.exp((r - 0.5*sigma**2)*dt + sigma*np.sqrt(dt)*Z)
    
    # Calculate option payoffs
    payoffs = np.maximum(S_t - K, 0)
    
    # Calculate option price using Monte Carlo simulation
    discount_factor = np.exp(-r*T)
    option_price = discount_factor * np.mean(payoffs)
    
    # Calculate delta using likelihood ratio method
    delta_likelihood_ratio = np.mean(payoffs * norm.cdf(d1)) / S
    
    return option
